In [4]:
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import time
import glob
from lxml.html import parse
from sphfile import SPHFile
import pydub
import audiosegment
import pandas as pd
from collections import Counter
from bs4 import BeautifulSoup
import sys
root = '../'

## Split

In [6]:
segments_df = pd.read_csv('files/segments.csv')
segments_df = segments_df.sample(frac=1, random_state = 12345)

speakers = Counter(segments_df['speaker'])
common_speakers = ['andrea_arsenault', 'brian_lamb', 'csp_waj_susan', 'david_brancaccio', 'eddie_mair', 'joie_chen', 'kathleen_kennedy', 'leon_harris', 'linda_wertheimer', 'linden_soles', 'lisa_mullins', 'lou_waters', 'lynn_vaughan', 'mark_mullen', 'natalie_allen', 'noah_adams', 'peter_jennings', 'robert_siegel', 'ted_koppel', 'thalia_assuras']
#common_speakers = speakers.most_common(20)
common_speakers = [(common_speaker, speakers[common_speaker]) for common_speaker in common_speakers]
print(common_speakers)
#common_speakers = [common_speaker[0] for common_speaker in common_speakers]
segments_by_speakers = {speaker:segments_df[segments_df['speaker']==speaker] for speaker, _ in common_speakers}
segments_df

[('andrea_arsenault', 1002), ('brian_lamb', 2590), ('csp_waj_susan', 1371), ('david_brancaccio', 2872), ('eddie_mair', 1346), ('joie_chen', 5360), ('kathleen_kennedy', 1778), ('leon_harris', 3099), ('linda_wertheimer', 1368), ('linden_soles', 3354), ('lisa_mullins', 1443), ('lou_waters', 3838), ('lynn_vaughan', 1887), ('mark_mullen', 1483), ('natalie_allen', 2750), ('noah_adams', 2018), ('peter_jennings', 5000), ('robert_siegel', 1184), ('ted_koppel', 2783), ('thalia_assuras', 1587)]


,silence_ratio,speaker,segfile
125298,0.0,lou_waters,files/segments/e960514b_turn69_seg6.npy
249225,0.0,tony_blankley,files/segments/a960610__turn56_seg27.npy
158478,0.0,linda_wertheimer,files/segments/j960521a_turn62_seg9.npy
137331,0.0,bill_maynes,files/segments/i960531__turn427_seg9.npy
214924,0.0,david_brancaccio,files/segments/k960610__turn40_seg4.npy
...,...,...,...
85412,0.0,valerie_morris,files/segments/ed980122_turn36_seg8.npy
133249,0.0,i960604_m_us_027,files/segments/i960604__turn434_seg2.npy
130333,0.0,mguest4_native,files/segments/h960514__turn168_seg0.npy
77285,0.0,steve_coz,files/segments/eo970830_turn64_seg13.npy


In [7]:
train_df = pd.DataFrame()
for speaker, num_segments in common_speakers:
    tenth = int(num_segments*0.1)
    spkr_train_segments = segments_by_speakers[speaker][:tenth*8]
    train_df = train_df.append(spkr_train_segments)
train_df = train_df.sample(frac=1, random_state = 12345)
train_df.to_csv('train-segments.csv', index = False)
train_df

,silence_ratio,speaker,segfile
176269,0.0,csp_waj_susan,files/segments/i960606__turn389_seg0.npy
190651,0.0,thalia_assuras,files/segments/b960613b_turn22_seg0.npy
118499,0.0,david_brancaccio,files/segments/file4_turn6_seg10.npy
195916,0.0,kathleen_kennedy,files/segments/h960517__turn80_seg7.npy
220309,0.0,lynn_vaughan,files/segments/f960607a_turn40_seg10.npy
...,...,...,...
212033,0.0,david_brancaccio,files/segments/k960613__turn39_seg11.npy
61427,0.0,lisa_mullins,files/segments/eh971015_turn36_seg8.npy
146339,0.0,brian_lamb,files/segments/i960610__turn0_seg1.npy
194517,0.0,kathleen_kennedy,files/segments/h960516__turn110_seg4.npy


In [8]:
val_df = pd.DataFrame()
for speaker, num_segments in common_speakers:
    tenth = int(num_segments*0.1)
    spkr_val_segments = segments_by_speakers[speaker][tenth*8:tenth*9]
    val_df = val_df.append(spkr_val_segments)
val_df = val_df.sample(frac=1, random_state = 12345)
val_df.to_csv('val-segments.csv')
val_df

,silence_ratio,speaker,segfile
162515,0.0,natalie_allen,files/segments/e960517a_turn1_seg0.npy
44149,0.0,lou_waters,files/segments/ee970807_turn1_seg2.npy
8755,0.0,eddie_mair,files/segments/eh971027_turn64_seg4.npy
194986,0.0,natalie_allen,files/segments/g960611__turn21_seg0.npy
149634,0.0,brian_lamb,files/segments/i960610__turn531_seg8.npy
...,...,...,...
127684,0.0,natalie_allen,files/segments/e960513b_turn37_seg13.npy
101754,0.0,natalie_allen,files/segments/ee970722_turn77_seg0.npy
196863,0.0,linden_soles,files/segments/g960524__turn53_seg7.npy
238535,0.0,noah_adams,files/segments/j960614c_turn1_seg2.npy


In [9]:
test_df = pd.DataFrame()
for speaker, num_segments in common_speakers:
    tenth = int(num_segments*0.1)
    spkr_test_segments = segments_by_speakers[speaker][tenth*9:]
    test_df = test_df.append(spkr_test_segments)
test_df = test_df.sample(frac=1, random_state = 12345)
test_df.to_csv('test-segments.csv')
test_df

,silence_ratio,speaker,segfile
115157,0.0,brian_lamb,files/segments/ew970627_turn101_seg4.npy
184430,0.0,linden_soles,files/segments/g960516__turn43_seg3.npy
186067,0.0,lynn_vaughan,files/segments/f960614__turn15_seg3.npy
193048,0.0,mark_mullen,files/segments/b960618__turn41_seg10.npy
194411,0.0,linden_soles,files/segments/h960516__turn96_seg6.npy
...,...,...,...
197344,0.0,natalie_allen,files/segments/g960607__turn37_seg1.npy
127547,0.0,natalie_allen,files/segments/e960513b_turn23_seg9.npy
194426,0.0,linden_soles,files/segments/h960516__turn98_seg5.npy
45912,0.0,natalie_allen,files/segments/ee970806_turn49_seg6.npy
